In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("DailyDelhiClimateTrain.csv")
df2 = pd.read_csv("DailyDelhiClimateTest.csv")


In [3]:
df = pd.concat([df1 , df2] ,axis=0)

In [4]:
df.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [8]:
df['MA_for_250_days'] = df['humidity'].rolling(100).mean()

In [10]:
df['MA_for_250_days'][0:100].tail()

95          NaN
96          NaN
97          NaN
98          NaN
99    64.185071
Name: MA_for_250_days, dtype: float64

In [12]:
df['percentage_change_cp'] = df['humidity'].pct_change()
df[['humidity','percentage_change_cp']].head()

,humidity,percentage_change_cp
0,84.500000,NaN
1,92.000000,0.088757
2,87.000000,-0.054348
3,71.333333,-0.180077
4,86.833333,0.217290


In [13]:
humidity = df[['humidity']]

In [15]:
max(humidity.values),min(humidity.values) 

(array([100.]), array([13.42857143]))

In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(humidity)
scaled_data

array([[0.8209571 ],
       [0.90759076],
       [0.84983498],
       ...,
       [0.31732673],
       [0.16254125],
       [0.15841584]])

In [18]:
len(scaled_data)

1576

In [19]:
x_data = []
y_data = []

for i in range(100, len(scaled_data)):
    x_data.append(scaled_data[i-100:i])
    y_data.append(scaled_data[i])
    
import numpy as np
x_data, y_data = np.array(x_data), np.array(y_data)

In [20]:
splitting_len = int(len(x_data)*0.7)
x_train = x_data[:splitting_len]
y_train = y_data[:splitting_len]

x_test = x_data[splitting_len:]
y_test = y_data[splitting_len:]

In [21]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [22]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(64,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

d:\Installation\Python\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [24]:
model.fit(x_train, y_train, batch_size=1, epochs = 2)

Epoch 1/2
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - loss: 0.0219
Epoch 2/2
1033/1033 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - loss: 0.0102


In [25]:
predictions = model.predict(x_test)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step


In [26]:
inv_predictions = scaler.inverse_transform(predictions)

array([[72.67056 ],
       [63.907776],
       [65.34472 ],
       [63.275723],
       [55.818703],
       [58.631363],
       [61.54275 ],
       [60.853806],
       [58.806637],
       [56.18883 ],
       [58.57058 ],
       [71.43743 ],
       [65.105064],
       [68.32391 ],
       [58.903458],
       [52.817814],
       [52.621593],
       [58.141502],
       [57.144634],
       [55.44513 ],
       [57.85597 ],
       [60.63201 ],
       [60.958664],
       [55.953075],
       [56.086124],
       [55.029453],
       [46.689606],
       [56.293564],
       [63.083607],
       [60.441437],
       [57.671494],
       [56.0105  ],
       [56.167168],
       [64.270706],
       [62.89753 ],
       [61.807087],
       [62.758865],
       [57.479515],
       [54.155457],
       [51.79659 ],
       [60.091557],
       [59.852406],
       [57.29252 ],
       [46.05903 ],
       [42.07336 ],
       [45.592022],
       [47.96439 ],
       [49.9647  ],
       [58.990498],
       [51.527794],


In [27]:
inv_y_test = scaler.inverse_transform(y_test)

In [28]:
rmse = np.sqrt(np.mean( (inv_predictions - inv_y_test)**2))

In [29]:
rmse

7.368202202968044